# Financial Analysis of Facebook Prices
This analysis will examine the stock prices of Facebook from 2012 to 2021. The data was obtained from Kaggle and will be used to gain insights into the performance of the company over the past decade.We will explore trends, patterns, and relationships in the data to better understand the financial performance of the tech giant.

In [ ]:
import datetime as dt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pandas_ta as ta
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
fb=pd.read_csv('../input/facebook-stock-data-live-and-latest/FB_stock_history.csv')
fb.head

# Data Processing
## Facebook Dataset
The Facebook dataset contains 2359 rows and 8 columns.
The columns in the dataset are: 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', and 'Stock Splits'.
There are no null values or duplicates in the dataset.

In [ ]:
fb.isnull().sum()

In [ ]:
fb.shape

In [ ]:
fb.drop_duplicates()

In [ ]:
fb.columns

# Stock Technical Analysis 
We will be studying historical price and volume data to identify patterns and trends that we will use to predict future price movements


In [ ]:
fb['Price1'] = fb['Close'].shift(-1)
fb #added a new column to the fb dataframe price 1

In [ ]:
fb['Pricediff']=fb['Price1']-fb['Close']
fb #added another column by finding the price difference

## Daily Returns
In stock analysis, the daily return of a stock is the percentage change in its price from one day to the next. It is calculated by taking the difference between the closing price of a stock on two consecutive days, dividing that difference by the closing price on the first day, and then multiply by 100 to express in percentage.

In [ ]:
fb['Daily_Returns']=fb['Pricediff']/fb['Close']
fb

## Direction
In stock analysis, direction refers to the trend of a stock’s price movement. Technical analysts and chartists seek to identify patterns in stock charts to anticipate the future direction of a security’s price. Direction typically refers to the trend of a stock’s price movement. There are several ways to calculate the direction or trend of a stock’s price using Pandas. One common method is to calculate a moving average of the stock’s price and use it as a trend indicator.

### A Moving Average as a Trend Indicator
In stock analysis, a moving average is commonly used as a trend indicator. A shorter period moving average, such as a 40-day moving average, is considered a fast signal because it reacts more quickly to recent price changes. On the other hand, a longer period moving average, such as a 200-day moving average, is considered a slow signal because it takes longer to react to price changes and tends to smooth out short-term fluctuations.

In [ ]:
fb['MA_20']=fb['Close'].rolling(20).mean()
fb['MA_20'].plot()

In [ ]:
fb['MA_20']=fb['Close'].rolling(20).mean()
fb['MA_50']=fb['Close'].rolling(50).mean()
fb['MA_200']=fb['Close'].rolling(200).mean()
fb[['MA_20','MA_50','MA_200']].plot()

## Exponential Moving Average (EMA)
EMA stands for Exponential Moving Average. It is a type of moving average that places greater weight on more recent data points. This makes it more responsive to recent price changes.
### What is the difference btwn SMA and EMA
The main difference between an Exponential Moving Average (EMA) and a Simple Moving Average (SMA) according to [investopedia](https://www.investopedia.com/ask/answers/difference-between-simple-exponential-moving-average/) is the sensitivity each one shows to changes in the data used in its calculation. The EMA gives a higher weighting to recent prices, while the SMA assigns equal weighting to all values

In [ ]:
fb['EMA_20']=ta.ema(fb['Close'], 20)
plt.plot(fb['EMA_20'], label=['EMA_20'])
plt.legend()
plt.show()

## Relative strenght index

RSI stands for Relative Strength Index. It is a momentum indicator used in technical analysis to measure the speed and magnitude of a security’s recent price changes. The RSI is displayed as an oscillator on a scale of 0 to 100, with values [above 70 indicating overbought conditions and values below 30](https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/RSI#:~:text=The%20Relative%20Strength%20Index%20(RSI,and%20oversold%20when%20below%2030.) indicating oversold conditions

In [ ]:
fb['RSI_14'] = ta.rsi(fb['Close'], length=14)
plt.plot(fb['RSI_14'], label='14-day RSI')
plt.axhline(30, color='red', linestyle='--')
plt.axhline(70, color='green', linestyle='--')
plt.legend()
plt.show()

## Cumulative Wealth
[Cumulative wealth](https://thismatter.com/money/investments/investment-math.htm) is a measure of the total amount of wealth accumulated over a period of time.In stock analysis, it represents the total value of an investment at a given point in time, taking into account all the changes in stock prices and any dividends paid.

In [ ]:
fb['Cumulative Wealth'] = (1 + fb['Daily_Returns']).cumprod()
fb['Cumulative Wealth'].plot()
plt.title('Cumulative Wealth Over Time')
plt.xlabel('Date')
plt.ylabel('Cumulative Wealth')
plt.show()

## Distribution of long return
The distribution of long-term returns in stock analysis refers to the statistical calculation that describes the chance that a given stock’s return will fall between or within a specific range on a plotting chart. This is known as a probability distribution.

In [ ]:
fb['LogReturn'] = np.log(fb['Close']).shift(-1) - np.log(fb['Close'])

In [ ]:
from scipy.stats import norm
mu = fb['LogReturn'].mean()
sigma = fb['LogReturn'].std(ddof=1)

density = pd.DataFrame()
density['x'] = np.arange(fb['LogReturn'].min()-0.01, fb['LogReturn'].max()+0.01, 0.001)
density['pdf'] = norm.pdf(density['x'], mu, sigma)

fb['LogReturn'].hist(bins=50, figsize=(15, 8))
plt.plot(density['x'], density['pdf'], color='red')
plt.show()

## Calculating the probability of the stock price dropping over a certain percentage in a day

In [ ]:
# probability that the stock price of facebook will drop over 20% in a day
prob_return1 = norm.cdf(-0.20, mu, sigma)
print('The Probability is ', prob_return1)

## Calculating the probability of the stock price dropping over a certain percentage in a year

In [ ]:
# drop over 40% in 365 days
mu220 = 220*mu
sigma220 = (220**0.5) * sigma
print('The probability of dropping over 40% in 220 days is ', norm.cdf(-0.4, mu220, sigma220))

## Calculating Value at risk (VaR)
Value at Risk (VaR) is a statistical measure that quantifies the level of financial risk within a firm, portfolio or position over a specific time frame. It is commonly used by investment and commercial banks to determine the extent and occurrence probability of potential losses in their institutional portfolios. Risk managers use VaR to measure and control the [level of risk exposure](https://www.investopedia.com/articles/04/092904.asp)

In [ ]:
VaR = norm.ppf(0.05, mu, sigma)
print('Single day value at risk ', VaR)

In [ ]:
# Quantiles
q25= norm.ppf(0.25,mu,sigma)
print('25th percentile',q25)
q50= norm.ppf(0.50,mu,sigma)
print('50th percentile',q50)
q75= norm.ppf(0.75,mu,sigma)
print('75th percentile',q75)

# Simple Linear Reggression Prediction Model

In [ ]:
from sklearn.linear_model import LinearRegression
X= fb[['Volume']]
y= fb[['Close']]
model = LinearRegression()
model.fit(X, y)
predictions = model.predict(X)
score = model.score(X, y)
print(f'R-squared: {score:.2f}')
plt.scatter(X, y, color='blue')
plt.plot(X, predictions, color='red', linewidth=2)
plt.title('FB Stock Prices')
plt.xlabel('Volume')
plt.ylabel('Close')
plt.show()


*  The distribution of the stock’s long return was approximately normal. 
*  The probability of the stock price dropping by a certain percentage in a single day was (1.2399894408713814e-18), 
*  The probability of it dropping by that percentage in a year was 0.03820026587335036. 
*  The single day value at risk is -0.036914579239954406.
*  The moving averages (MA_20, MA_50, and MA_200) as well as the exponential moving average (EMA) showed an upward trend.
*  The cumulative wealth over time also showed an upward trend.
 